<a href="https://colab.research.google.com/github/Gorphe/Howard-Gugger-Book/blob/main/7_state_of_the_art_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from fastai.vision.all import *

In [6]:
path = untar_data(URLs.IMAGENETTE)

In [7]:
dblock = DataBlock(
    blocks = (ImageBlock(), CategoryBlock()),
    get_items = get_image_files,
    get_y=parent_label,
    item_tfms = Resize(460),
    batch_tfms = aug_transforms(size=224, min_scale=0.75)
)

dls = dblock.dataloaders(path, bs=64)

In [8]:
model = xresnet50(n_out=dls.c)

learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5,3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.537085,2.791035,0.368932,03:02
1,1.217463,2.118341,0.467886,03:11
2,0.945634,1.294794,0.589246,03:00
3,0.719614,0.706802,0.778193,02:42
4,0.602383,0.566170,0.822255,02:41


NORMALIZATION

In [9]:
x,y = dls.one_batch()
x.mean(dim=[0,2,3]),x.std(dim=[0,2,3])

(TensorImage([0.4948, 0.4768, 0.4324], device='cuda:0'),
 TensorImage([0.2782, 0.2770, 0.2982], device='cuda:0'))

In [10]:
def get_dls(bs, size):
  dblock = DataBlock(
     blocks=(ImageBlock, CategoryBlock),
     get_items=get_image_files,
     get_y=parent_label,
     item_tfms=Resize(460),
     batch_tfms=[*aug_transforms(size=size, min_scale=0.75), Normalize.from_stats(*imagenet_stats)]
  )

  return dblock.dataloaders(path, bs=bs)

dls = get_dls(64,224)

x, y = dls.one_batch()
x.mean(dim=[0,2,3]), x.std(dim=[0,2,3])

(TensorImage([-0.0620, -0.0541,  0.0213], device='cuda:0'),
 TensorImage([1.2486, 1.2301, 1.2735], device='cuda:0'))

In [11]:
model = xresnet50()
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5,3e-3)
                

epoch,train_loss,valid_loss,accuracy,time
0,1.662048,1.941448,0.426811,02:41
1,1.247668,3.002167,0.410381,02:41
2,0.970994,1.102357,0.658701,02:40
3,0.749676,0.713774,0.793129,02:40
4,0.606217,0.591358,0.820015,02:41


PROGRESSIVE RESIZING

In [ ]:
dls = get_dls(128,128)
learn = Learner(dls, xresnet50(n_out=dls.c), loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(4,3e-3)


epoch,train_loss,valid_loss,accuracy,time
0,1.571288,2.217920,0.432786,02:15
1,1.211564,1.100399,0.659074,02:15
2,0.928153,0.832985,0.734503,02:15


In [1]:
learn.dls = get_dls(64,224)
learn.fine_tune(5,1e-3)

NameError: ignored

TEST TIME AUGMENTATION

In [ ]:
preds, targs = learn.tta()
accuracy(preds, targs).item()

MIXUP

In [ ]:
model = xresnet50(n_out=dls.c)
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy, cbs=MixUp())
learn.fit_one_cycle(5,3e-3)

LABEL SMOOTHING

In [ ]:
model = xresnet50(n_out=dls.c)
learn = Learner(dls, model, loss_func=LabelSmoothingCrossEntropy(),metrics=accuracy)
learn.fit_one_cycle(5,3e-3)